# 04 Optimize

In [1]:
import pandas as pd
import numpy as np
import sys
sys.path.append('../src')
from optimize import optimize_tourney

In [2]:
df = pd.read_csv('../data/opt_input.csv')
df.head()

,Season,Slot,StrongSeed,WeakSeed,Round,TeamID_Strong,TeamName_Strong,TeamID_Weak,TeamName_Weak,TeamID_Winner,Strong_Win,Orig_Seed_Strong,Orig_Seed_Weak,Matchup,Round_v2,comp,Key,Win_Prob
0,2018,R1W1,W01,W16,1,1437,Villanova,1347,Radford,1437,1,1,16,15,1,1,115,0.992647
1,2018,R1W1,W01,W16,1,1437,Villanova,1347,Radford,1347,0,1,16,15,1,-1,115,0.007353
2,2018,R2W1,R1W1,R1W8,2,1437,Villanova,1439,Virginia Tech,1437,1,1,8,7,2,1,207,0.811594
3,2018,R2W1,R1W1,R1W8,2,1437,Villanova,1439,Virginia Tech,1439,0,1,8,7,2,-1,207,0.188406
4,2018,R2W1,R1W1,R1W8,2,1347,Radford,1439,Virginia Tech,1347,1,16,8,8,2,-1,208,0.254310


In [3]:
for c in set(df['Slot']):
    df.loc[df['Slot']==c, 'Round_Game_'+c] = 1
    df['Round_Game_'+c].fillna(0, inplace=True)

In [4]:
teams = set(list(df['TeamID_Strong']) + list(df['TeamID_Weak']))

In [5]:
def set_value(rnd, win_tm, w_tm, s_tm, r, t):
    if rnd==r:
        if (w_tm==t or s_tm==t):
            return r
    if rnd<r:
        if win_tm==t:
            return -1
    return 0
set_value_v = np.vectorize(set_value, excluded=['r', 't'])

for r in set(df['Round']):
    for t in teams:
        df[str(t)+'_'+str(r)] = set_value_v(df['Round'], df['TeamID_Winner'], df['TeamID_Weak'], df['TeamID_Strong'], r, t)

In [6]:
opt_results = optimize_tourney(df, 'Win_Prob', 'max')

In [7]:
opt_results.head()

,Season,Slot,StrongSeed,WeakSeed,Round,TeamID_Strong,TeamName_Strong,TeamID_Weak,TeamName_Weak,TeamID_Winner,...,1395_6,1138_6,1397_6,1400_6,1401_6,1274_6,1403_6,1276_6,1277_6,Chosen
0,2018,R1W1,W01,W16,1,1437,Villanova,1347,Radford,1437,...,0,0,0,0,0,0,0,0,0,1.0
2,2018,R2W1,R1W1,R1W8,2,1437,Villanova,1439,Virginia Tech,1437,...,0,0,0,0,0,0,0,0,0,1.0
8,2018,R1W8,W08,W09,1,1439,Virginia Tech,1104,Alabama,1439,...,0,0,0,0,0,0,0,0,0,1.0
23,2018,R1W4,W04,W13,1,1455,Wichita St,1267,Marshall,1267,...,0,0,0,0,0,0,0,0,0,1.0
42,2018,R3W1,R2W1,R2W4,3,1437,Villanova,1293,Murray St,1437,...,0,0,0,0,0,0,0,0,0,1.0


In [8]:
opt_results.to_csv('../data/opt_results.csv', index=False)

In [9]:
df.to_csv('../data/opt_input_v2.csv', index=False)